<span style="font-size: 1.5em;">Solving 8-Puzzle with RBFS </span>
---
<hr style="border: none; border-top: 3px solid black;">

## ◎ **$\texttt{Description}$** 

#### In this question the 8-Puzzle game is solved using the RBFS (Recursive Best-First Search) informed search algorithm.
>You are asked to complete the heuristic and rbfs functions so that, ultimately, for different initial states of the puzzle, we can reach the final state where the numbers 1 to 8 are arranged in order.

In [1]:
import sys 
import time

# ◎ **$\texttt{heuristic}$** 

>As you can see we assume **MANHATTAN DISTANCE** as our heuristic funcion.<br>
Also various heuristic function for this case such like pattern database can be used.

In [2]:
def heuristic(state, goal_state):
    '''the summation of manhattan distances between each tile and the correct position of it '''
    
    distance = 0
    for i in range(3): 
        for j in range(3):
            value = state[i][j]
            
            if value != 0:
                goal_i, goal_j = divmod(value - 1, 3)
                distance += abs(i - goal_i) + abs(j - goal_j)
    return distance

## ◎ **$\texttt{generate_successors}$**
---
>This function takes a 3x3 matrix $\texttt{state}$ as input and generates a list of successor states  by moving the empty cell (indicated by a 0) <br> in all possible directions (up, down, left, right). The function returns a list of successor states.

###  $\text{I.}$  $\texttt{Parameters}$
- `state` : list
    - A 3x3 matrix representing the current state of the puzzle.

### $\text{II.}$  $\texttt{Returns}$ 
- `successors` : list
    - A list of 3x3 matrices, each representing a possible successor state of the puzzle.
    
### $\text{III.}$  $\texttt{Algorithm}$
1. Initialize an empty list (successors).
2. Find the indices of the empty cell in the state matrix.
3. Define a list moves containing the possible movements of the empty cell.
4. For each move in moves, calculate the new indices of the empty cell.
5. If the new indices are within the bounds of the matrix, create a new state by swapping the empty cell with the adjacent cell in the direction of the move.
6. Append the new state to the list of successors.
7. Return the list of successors.





In [3]:
def generate_successors(state):
    successors = []
    empty_i, empty_j = None, None
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                empty_i, empty_j = i, j

    moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    for move in moves:
        new_i, new_j = empty_i + move[0], empty_j + move[1]
        if 0 <= new_i < 3 and 0 <= new_j < 3:
            new_state = [row[:] for row in state]
            new_state[empty_i][empty_j] = new_state[new_i][new_j]
            new_state[new_i][new_j] = 0
            successors.append(new_state)

    return successors


In [4]:
def is_goal(state, goal_state):
    return state == goal_state


In [5]:
# Print puzzle 
def print_puzzle(state):
    for row in state:
        print(row)

## ◎ **$\texttt{RBFS}$**
---
>This function implements the Recursive Best-First Search (RBFS) algorithm to find the optimal path from a given **state** to the **goal_state**. The function returns the optimal path and its cost.

###  $\text{I.}$  $\texttt{Parameters}$
- `state` : list
    - A 2-dimensional list representing the current state of the puzzle.
- `goal_state` : list
    - A 2-dimensional list representing the goal state of the puzzle.
- `f_limit` : int
    - The maximum cost of a path that the algorithm is allowed to explore.
- `g_cost` : int (default: 0)
    - The cost of the current path.
- `timeout` : int (default: 10)
    - The maximum amount of time (in seconds) that the algorithm is allowed to run.

### $\text{II.}$  $\texttt{Returns}$ 
- `result` : list
    - A list representing the optimal path from the current state to the goal state.
- `best_f` : int
    - The cost of the optimal path.

### $\text{III.}$  $\texttt{Algorithm}$
1. If the state is the same as the *goal_state*, return the state and the cost of the current path, which is 0.
2. Generate all possible successor states of the state using the `generate_successors` function.
3. If there are no successors, return None and a large number (i.e. infinity).
4. Sort the successor states by their estimated cost (i.e. *g_cost* + 1 + *heuristic*).
5. If the estimated cost of the best successor state is greater than the f_limit, return None and the estimated cost of the best successor.
6. If there is more than one successor state, set alternative to the estimated cost of the second best successor, otherwise set it to infinity.
7. Calculate the remaining time for the algorithm to run, which is the timeout minus the time already elapsed.
8. Recursively call the `RBFS` function with the best successor state as the new state
9. Update f_limit to be the minimum of f_limit and *alternative*, and *g_cost* to be *g_cost* + 1.
10. If the remaining time is less than or equal to 0, return None and a large number (i.e. infinity).
11. If the recursive call returns a result, return the result and the cost of the optimal path.
12. If the remaining time is less than or equal to 0, return None and a large number (i.e. infinity).

In [6]:
def rbfs(state, goal_state, f_limit, g_cost=0, timeout=10):
    if is_goal(state, goal_state):
        return state, g_cost

    successors = [(s, g_cost + 1 + heuristic(s, goal_state)) for s in generate_successors(state)]
    if not successors:
        return None, sys.maxsize

    start_time = time.time()

    while True:
        successors.sort(key=lambda x: x[1])
        best_state, best_f = successors[0]
        if best_f > f_limit:
            return None, best_f

        alternative = successors[1][1] if len(successors) > 1 else sys.maxsize
        remaining_time = (timeout - (time.time() - start_time))

        result, best_f = rbfs(best_state, goal_state, min(f_limit, alternative), g_cost + 1, timeout=remaining_time)

        if time.time() - start_time >= timeout:
            return None, sys.maxsize

        if result is not None:
            return result, best_f

        if time.time() - start_time >= timeout:
            return None, sys.maxsize
        

In [9]:
initial_state =  [[2, 3, 0], [1, 4, 6], [7, 5, 8]]
#initial_state =[[1, 2, 3], [8, 0, 4], [7, 6, 5]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

f_limit = heuristic(initial_state, goal_state)
result, f_goal = rbfs(initial_state, goal_state, f_limit)

print("Initial state:")
print_puzzle(initial_state)

if result is not None:
    print("Goal state reached with {} step(s):".format(f_goal))
    print_puzzle(result)
else:
    print("Goal state could not be reached.")

Initial state:
[2, 3, 0]
[1, 4, 6]
[7, 5, 8]
Goal state reached with 6 step(s):
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
